In [ ]:
# SK KoBERT에서 요구하는 패키지
# boto3 <=1.15.18
# gluonnlp >= 0.6.0, <=0.10.0
# mxnet >= 1.4.0, <=1.7.0.post2
# onnxruntime == 1.8.0, <=1.8.0
# sentencepiece >= 0.1.6, <=0.1.96
# torch >= 1.7.0, <=1.10.1
# transformers >= 4.8.1, <=4.8.1

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

In [52]:
data = pd.read_csv('./train.csv')[['overview', 'cat3']]

In [3]:
data.shape

(16986, 2)

In [4]:
data.head()

,overview,cat3
0,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,항구/포구
1,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,골프
2,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,한식
3,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,한식
4,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,한식


In [5]:
data_list = []
for ques, label in zip(data['overview'], data['cat3']):
    tmp = []   
    tmp.append(ques)
    tmp.append(str(label))

    data_list.append(tmp)

In [6]:
data_list[0]

['소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 올리고 있으며 바다낚시터로도 유명하다. 항 주변에 설치된 양식장들은 섬사람들의 부지런한 생활상을 고스 란히 담고 있으며 일몰 때 섬의 정경은 바다의 아름다움을 그대로 품고 있는 듯하다. 또한, 섬에는 각시여 전설, 도둑바위 등의 설화가 전해 내려오고 있으며, 매년 정월 풍어제 풍속이 이어지고 있다.<br>',
 '항구/포구']

In [7]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [8]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-tk7ao8nr
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-tk7ao8nr
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3


In [9]:
# kobert 모듈 로드
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model


In [20]:
# torch, nlp를 위한 모듈 로드
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np

from datetime import datetime
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import re
import gc
import os

# kobert 모듈 로드
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# transformer 모듈 로드
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

# argument
import argparse


# 학습 데이터셋 구축 클래스
class BERTDataset(Dataset):
    
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

    
# BERT 분류기 클래스
class BERTClassifier(nn.Module):
    
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 128, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
#         print(attention_mask)
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out), pooler    
    

# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [21]:
bertmodel, vocab = get_pytorch_kobert_model()
tokenizer = get_tokenizer()

using cached model. /home/rok/Multimodal_Clf/Multimodal-Clf/.cache/kobert_v1.zip
using cached model. /home/rok/Multimodal_Clf/Multimodal-Clf/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/rok/Multimodal_Clf/Multimodal-Clf/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [22]:
device = torch.device("cuda:0") #GPU 사용 시
#device = torch.device("cpu") #CPU 사용 시
torch.cuda.is_available()

# 토크나이저 생성
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

model = BERTClassifier(bertmodel, dr_rate=0.2).to(device)
model = nn.DataParallel(model, device_ids=[0,1])
model.cuda()

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [23]:
data.head()

,overview,cat3
0,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,120
1,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,8
2,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,118
3,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,118
4,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,118


In [24]:
data_list[0]

['소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 올리고 있으며 바다낚시터로도 유명하다. 항 주변에 설치된 양식장들은 섬사람들의 부지런한 생활상을 고스 란히 담고 있으며 일몰 때 섬의 정경은 바다의 아름다움을 그대로 품고 있는 듯하다. 또한, 섬에는 각시여 전설, 도둑바위 등의 설화가 전해 내려오고 있으며, 매년 정월 풍어제 풍속이 이어지고 있다.<br>',
 '항구/포구']

In [53]:
from sklearn import preprocessing
  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
data['cat3']= label_encoder.fit_transform(data['cat3'])
  
data['cat3'].unique()

array([120,   8, 118,  73,  58,  90,  85,  91,  94,  97,  12,  31,  67,
        41, 101, 119, 105,  59, 100,  72,  96,  86, 121,   0,  37,  11,
        54,  44,  25,   9,  20,  48, 115,  53,  16,  15,  92,  95, 127,
        82,  61,  84,  52,   3,  93,  43,  35,   5, 116,  57,  99,  27,
        19,   7,  13,   4,  71,  79,  34,  29,  69,  10, 112,  77,  60,
       122, 114,  17,  18, 126,  38, 107,  39, 111,  81,  42,  63,  46,
       125, 102,  78,  30,  21, 103,  49,  26,  24,  80, 113,  83,   6,
        56,  40,  98,  32,  70,  47, 110,  33, 104, 109,  45,  22,   2,
        14,  66,  23,  55,  65,  76, 124,  64,  74,  89,  87, 117, 123,
        28,  51,   1,  50,  62,  75, 106,  68,  36,  88, 108])

In [71]:
num_to_chr = {}
for i in data['cat3'].unique():
    num_to_chr[i] = label_encoder.inverse_transform(np.array([i]))[0]

In [72]:
num_to_chr

{120: '항구/포구',
 8: '골프',
 118: '한식',
 73: '야영장,오토캠핑장',
 58: '섬',
 90: '일반축제',
 85: '유적지/사적지',
 91: '일식',
 94: '자연휴양림',
 97: '전시관',
 12: '관광단지',
 31: '모텔',
 67: '스키(보드) 렌탈샵',
 41: '바/까페',
 101: '채식전문점',
 119: '한옥스테이',
 105: '컨벤션',
 59: '성',
 100: '중식',
 72: '안보관광',
 96: '전문상가',
 86: '이색거리',
 121: '해수욕장',
 0: '5일장',
 37: '미술관/화랑',
 11: '공원',
 54: '상설시장',
 44: '박물관',
 25: '도서관',
 9: '공연장',
 20: '농.산.어촌 체험',
 48: '복합 레포츠',
 115: '펜션',
 53: '산',
 16: '기념탑/기념비/전망대',
 15: '기념관',
 92: '자동차경주',
 95: '자전거하이킹',
 127: '희귀동.식물',
 82: '유명건물',
 61: '수목원',
 84: '유원지',
 52: '사찰',
 3: '강',
 93: '자연생태관광지',
 43: '박람회',
 35: '문화전수시설',
 5: '계곡',
 116: '폭포',
 57: '서양식',
 99: '종교성지',
 27: '동상',
 19: '기타행사',
 7: '고택',
 13: '국립공원',
 4: '게스트하우스',
 71: '썰매장',
 79: '요트',
 34: '문화원',
 29: '래프팅',
 69: '승마',
 10: '공예,공방',
 112: '트래킹',
 77: '온천/욕장/스파',
 60: '수련시설',
 122: '해안절경',
 114: '패밀리레스토랑',
 17: '기암괴석',
 18: '기타',
 126: '홈스테이',
 38: '민물낚시',
 107: '콘도미니엄',
 39: '민박',
 111: '테마공원',
 81: '유람선/잠수함관광',
 42: '바다낚시',
 6

In [55]:
label_encoder.inverse_transform(np.array([120]))

array(['항구/포구'], dtype=object)

In [26]:
len(set(data['cat3'].tolist()))

128

In [28]:
data = data.rename(columns={'overview':'content', 'cat3':'label'})

In [29]:
data

,content,label
0,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,120
1,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,8
2,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,118
3,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,118
4,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,118
...,...,...
16981,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,118
16982,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,31
16983,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,31
16984,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,73


In [30]:
data_list=[]
for idx, row in tqdm(data.iterrows()):
    data_list.append([row[0], row[1]])


16986it [00:00, 19375.78it/s]


In [31]:
max_len = 64 #args['max_length']
batch_size = 64 #args['batch_size']
num_epochs = 10 # args['num_epochs']

In [32]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [33]:
# 학습
# Train / Test set 분리
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

# 학습을 위한 데이터셋 구축
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

# warm_up
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

# 스케쥴러
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [34]:

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out, hidden = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))


    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out, hidden = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


    # GPU 사용시 학습 전 캐시제거
    gc.collect()
    torch.cuda.empty_cache()




1it [00:03,  3.45s/it]

epoch 1 batch id 1 loss 4.867308139801025 train acc 0.0


201it [00:56,  3.75it/s]

epoch 1 batch id 201 loss 2.0619518756866455 train acc 0.3523009950248756


213it [00:59,  3.59it/s]

...

epoch 5 batch id 201 loss 0.7915986180305481 train acc 0.8335665422885572


213it [00:57,  3.72it/s]

epoch 5 train acc 0.8344776995305164



54it [00:05,  9.79it/s]


epoch 5 test acc 0.7573302469135803


In [35]:
# 모델 로컬 저장
torch.save(model, f'./hoac_result_m.pt')
torch.save(model.state_dict(), f'./hoac_result_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, f'./hoac_result.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능


In [36]:
# model load
model_loaded = torch.load(f'./hoac_result_m.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model_loaded.load_state_dict(torch.load(f'./hoac_result_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장
checkpoint = torch.load(f'./hoac_result.tar')   # dict 불러오기
model_loaded.load_state_dict(checkpoint['model'])


<All keys matched successfully>

In [43]:
# 예측값의 확률 값을 출력하기 위한 함수
def softmax(logits):
    argmax = torch.argmax(logits).item()
    sum_exp = torch.exp(logits).sum().item()
    argmax_exp = torch.exp(logits)[argmax].item()
    return argmax, argmax_exp/sum_exp, logits

In [37]:
def bert_predict(predict_sentence, thres = 0.5):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model_loaded.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out, sentence_vector = model_loaded(token_ids, valid_length, segment_ids)
#         print(out, sentence_vector)


        for i in out:

            argmax, prob, logits = softmax(i)

    return argmax, prob, logits # sentence_vector,

In [38]:
testset = pd.read_csv('./test.csv')

In [40]:
test_sentences = testset['overview'].tolist()

In [75]:

pred = []
for i in tqdm(test_sentences):
    pred.append(num_to_chr[bert_predict(i)[0]])
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 7280/7280 [1:06:05<00:00,  1.84it/s]


In [77]:
testset['cat3'] = pred
testset[['id', 'cat3']].to_csv('kobert_base.csv', index=False)